# 1. Jointures avec `JOIN`

## Principe
- Lier les données de différentes tables pour créer de nouvelles représentations tabulaires

-  Il existe différentes jointures :  pour les principaux : inner join, left join, right join, outer join.


![joints](../../images/joints.png)


**On utilise principalement left join, inner join, et outer join**

## Différentes jointures

### Left join
- Rejoindre la première table (gauche) avec une seconde (droite), même si certains enregistrements de la première table n'ont pas de correspondance dans la seconde

- Les commandes sans produits correspondants auront NULL pour product_name

Sur deux tables : 
```sql
SELECT 
  orders.order_id, 
  order_details.product_id
FROM 
  orders
LEFT JOIN order_details ON orders.order_id = order_details.order_id;
```

Sur trois tables : 
```sql
SELECT orders.order_id, products.product_name
FROM orders
LEFT JOIN order_details ON orders.order_id = order_details.order_id
LEFT JOIN products ON order_details.product_id = products.product_id ;
```

Avec une limite : 

```sql
SELECT 
  p1.product_name AS Product1, 
  p2.product_name AS Product2,
  p1.supplier_id
FROM 
  products p1
LEFT JOIN products p2 
  ON p1.supplier_id = p2.supplier_id AND p1.product_id <> p2.product_id
LIMIT 10;  -- For example, limits the result to 10 rows
```


### Inner join

- Ce code sélectionne les ID de commande et les noms de produits pour chaque commande

```sql
SELECT 
  orders.order_id, 
  order_details.product_id
FROM 
  orders
INNER JOIN order_details ON orders.order_id = order_details.order_id ;
```




### Full outer join

- Combinaison de left join et right join
- Sélectionner toutes les lignes des deux tables : inclut toutes les commandes et produits, même ceux sans correspondance

Sur deux tables : 
```sql
SELECT 
  orders.order_id, 
  order_details.product_id
FROM 
  orders
FULL OUTER JOIN order_details ON orders.order_id = order_details.order_id;
```

Sur trois tables : 
```sql
SELECT orders.order_id, products.product_name
FROM orders
FULL OUTER JOIN order_details ON orders.order_id = order_details.order_id
FULL OUTER JOIN products ON order_details.product_id = products.product_id ;
```


### Self join


#### Principe

- Un self-join associe la table products à elle-même pour comparer ou relier des données au sein de la même table
- Il faut utiliser des alias différents pour la même table pour différencier les instances

#### Exemple 1: Comparaison de produits similaires

- Supposons qu'on souhaite comparer des produits ayant le même category_id mais des product_id différents
- p1 et p2 sont des alias pour la table products
- Cette requête sélectionne des paires de produits dans la même catégorie.

```sql
SELECT 
  p1.product_name AS Product1, 
  p2.product_name AS Product2,
  p1.category_id
FROM 
  products p1
INNER JOIN products p2 
  ON p1.category_id = p2.category_id AND p1.product_id <> p2.product_id;

```

#### Exemple 2: Produits avec le même fournisseur

- Ici, le LEFT JOIN assure que tous les produits de p1 sont inclus, même s'il n'y a pas de produit correspondant dans p2

```sql
SELECT 
  p1.product_name AS Product1, 
  p2.product_name AS Product2,
  p1.supplier_id
FROM 
  products p1
LEFT JOIN products p2 
  ON p1.supplier_id = p2.supplier_id AND p1.product_id <> p2.product_id;
```




# 2. Aggregations avec `GROUPBY`

## Principe
- Diviser les lignes d'une requête en groupes puis appliquer des fonctions d'agrégation (comme SUM() ou COUNT()) sur chaque groupe


## Syntaxe

- Grouper les résultats par les valeurs des colonnes spécifiées dans GROUP BY
- Sélectionner des colonnes et appliquer une fonction d'agrégat sur une autre colonne

    
```sql
SELECT 
   colonne_1, 
   colonne_2, 
   fonction_agregat(colonne_3)
FROM 
   nom_table
GROUP BY 
   colonne_1, 
   colonne_2 ;
```

### Utiliser GROUP BY sans fonction d'agrégation
- Grouper les produits par category_id :
```sql
SELECT 
   category_id
FROM 
   products
GROUP BY 
   category_id ;
```
### Utiliser GROUP BY avec la fonction SUM()

- Calculer le prix total des stocks par catégorie :
```sql
SELECT
  category_id,
  SUM(unit_price * units_in_stock) AS total_stock_value
FROM
  products
GROUP BY
  category_id ;
```

### Utiliser GROUP BY avec la fonction COUNT()
- Compter le nombre de produits par catégorie :
```sql
SELECT
  category_id,
  COUNT(product_id)
FROM
  products
GROUP BY
  category_id ;
```  
### Utiliser GROUP BY avec plusieurs colonnes
- Total des stocks par catégorie et fournisseur :
```sql
SELECT 
  category_id, 
  supplier_id, 
  SUM(unit_price * units_in_stock) AS total_value
FROM 
  products
GROUP BY 
  category_id, 
  supplier_id ;
```  
### Utiliser GROUP BY avec une colonne de date
- Grouper et compter les commandes par date : 

```sql
SELECT 
  order_date , 
  COUNT(order_id)
FROM 
  orders
GROUP BY
  DATE(order_date) ;
```


### Utiliser GROUP BY avec JOIN
- Calculer le total des stocks par fournisseur :

```sql
SELECT
  s.company_name,
  p.supplier_id,
  SUM(p.unit_price * p.units_in_stock) AS total_value
FROM
  products p
JOIN suppliers s ON p.supplier_id = s.supplier_id
GROUP BY
  s.company_name, p.supplier_id;
```

# 4. Filtrage sur une aggrégation avec `HAVING`


## Principe
- Spécifier une condition de recherche pour un groupe ou un agrégat
- Souvent utilisée avec GROUP BY pour filtrer les groupes ou agrégats selon une condition spécifiée

## Syntaxe


- GROUP BY regroupe les lignes par colonne1.
- HAVING filtre ces groupes selon la condition spécifiée.


```sql
SELECT
  colonne1,
  fonction_agregat(colonne2)
FROM
  nom_table
GROUP BY
  colonne1
HAVING
  condition ;
```

**HAVING vs. WHERE**
- WHERE filtre les lignes selon une condition.
- HAVING filtre les groupes de lignes.

### Exemple : utiliser HAVING avec la fonction SUM
- Trouver les catégories dont le stock total dépasse une certaine valeur :
```sql
SELECT
  category_id,
  SUM(units_in_stock * unit_price) AS total_value
FROM
  products
GROUP BY
  category_id
HAVING
  SUM(units_in_stock * unit_price) > 10000 ;
```

### Exemple : utiliser HAVING avec la fonction COUNT
- Sélectionner les catégories ayant plus d'un certain nombre de produits :
```sql
SELECT
  category_id,
  COUNT(product_id)
FROM
  products
GROUP BY
  category_id
HAVING
  COUNT(product_id) > 5 ;
```

# 3. Opérations ensemblistes (sets)

## Principe
- Permet de combiner les résultats de plusieurs requêtes avec les opérations ensemblistes : UNION, INTERSECT, EXCEPT (=privé de), et de générer des groupes sous la forme de concaténations.
  

### UNION

- Combine les résultats de plusieurs requêtes en un seul ensemble de résultats

```sql
SELECT product_name 
FROM products 
WHERE category_id = 1
UNION
SELECT product_name 
FROM products 
WHERE category_id = 2 ;
```

### INTERSECT
- Retourne les lignes qui apparaissent dans les résultats de deux requêtes ou plus

```sql
SELECT product_name 
FROM products 
WHERE unit_price > 3
INTERSECT
SELECT product_name 
FROM products 
WHERE category_id = 1 ; 
```

### EXCEPT
Retourne les lignes de la première requête qui n'apparaissent pas dans les résultats de la seconde.

```sql
SELECT product_name 
FROM products 
WHERE supplier_id = 1
EXCEPT
SELECT product_name 
FROM products 
WHERE supplier_id = 2  ;
```

### Grouping Sets
- Permet de générer plusieurs ensembles de groupements dans un rapport

- Groupe les produits par category_id et supplier_id séparément : cette requête retourne la somme des prix unitaires pour chaque catégorie et chaque fournisseur séparément

```sql
SELECT category_id, supplier_id, SUM(unit_price)
FROM products
GROUP BY GROUPING SETS ((category_id), (supplier_id)) ;
```


###  Cube

- Défini plusieurs ensembles de groupements incluant toutes les combinaisons possibles de dimensions
- Groupe les produits par category_id et supplier_id en incluant toutes les combinaisons : cette requête retourne la somme des prix unitaires pour chaque catégorie, chaque fournisseur, ainsi que pour chaque combinaison de catégorie et fournisseur

```sql
SELECT category_id, supplier_id, SUM(unit_price)
FROM products
GROUP BY CUBE (category_id, supplier_id) ;
```
    
### Rollup
- Génére des rapports contenant des totaux et sous-totaux
- Groupe les produits par category_id et supplier_id en incluant des sous-totaux : cette requête retourne la somme des prix unitaires pour chaque catégorie, chaque fournisseur, ainsi que des sous-totaux pour chaque catégorie et un total général

```sql
SELECT category_id, supplier_id, SUM(unit_price)
FROM products
GROUP BY ROLLUP (category_id, supplier_id) ;
```




# 4. Ordre des clauses : 

Les clauses sont exécutées dans cet ordre : 

**`FROM` > `WHERE` > `GROUPBY` > `HAVING` > `SELECT` > `DISTINCT` > `ORDER BY`  > `LIMIT`**
